# Stock Prediction Framework Demo
### This notebook demonstrates the end-to-end functionality of the stock prediction system using predictor.py and models.py


In [ ]:
# # This is a sample code for the STA410 course at the University of Toronto.
# # It is not meant to be run as a standalone script.
# # It is meant to be run in a Jupyter notebook.
# # You can install the package usaing pip:
# %pip install stock-prediction-sta410

In [ ]:
# Import core modules
import matplotlib.pyplot as plt
from datetime import date
from sklearn.metrics import r2_score
import yfinance as yf

# Import custom modules
from stock_prediction.core import StockPredictor

# Energy sector companies
energy_sector = yf.Sector("energy").top_companies.index
technology_sector = yf.Sector("technology").top_companies.index
symbol= 'UBER' 

# Initialize predictor with example parameters
print(f"Selected symbol: {symbol}")
start_date = "2023-06-01"
end_date = date.today()

# Day trading
# predictor = StockPredictor(
#     symbol=symbol, start_date=date.today()-pd.Timedelta(days=1), end_date=date.today()+pd.Timedelta(days=1),  interval="1m"
# )

# Short term trading
predictor = StockPredictor(
    symbol=symbol, start_date=start_date, end_date=end_date, interval="1d"
)

In [ ]:
import requests
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce, QueryOrderStatus
trading_client = TradingClient('UR_API_KEY', 'UR_SECRET_KEY', paper=True)
trading_client.get_all_positions()

try:
    account = trading_client.get_account()
    print(f"Connected to {account.account_number}")
    print(f"Buying Power: {account.buying_power}")
    print(f"Trading Blocked: {account.trading_blocked}")
except Exception as e:
    print(f"Connection failed: {str(e)}")
    raise
account = trading_client.get_account()
float(account.equity)


In [ ]:
from alpaca.trading.enums import QueryOrderStatus
from alpaca.trading.requests import GetOrdersRequest
orders = trading_client.get_orders(filter=GetOrdersRequest(status=QueryOrderStatus.OPEN))
orders

In [ ]:
# GEt the available stock on the market
from alpaca.trading.requests import GetAssetsRequest
assets = trading_client.get_all_assets(filter=GetAssetsRequest(status="active", asset_class="us_equity"))
[asset for asset in assets if asset.symbol == 'AXP']


In [ ]:
trading_client.get_all_positions()

In [ ]:
from alpaca.data.historical import StockHistoricalDataClient, CryptoHistoricalDataClient
from alpaca.data.requests import CryptoSnapshotRequest
data_client = CryptoHistoricalDataClient(api_key='UR_API_KEY', secret_key='UR_SECRET_KEY')
req = CryptoSnapshotRequest(symbol_or_symbols='XRP/USD')
data_client.get_crypto_snapshot(req)

In [ ]:
from alpaca.data.requests import StockLatestTradeRequest
data_client2 = StockHistoricalDataClient(api_key='UR_API_KEY', secret_key='UR_SECRET_KEY')
data_client2.get_stock_latest_trade(StockLatestTradeRequest(symbol_or_symbols='AAPL'))['AAPL'].price

In [ ]:
list(set([stock.symbol for stock in trading_client.get_orders(
            filter=GetOrdersRequest(status=QueryOrderStatus.OPEN)
        )]))

In [ ]:
trading_client.get_orders(
            filter=GetOrdersRequest(status=QueryOrderStatus.OPEN)
        )

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Load and prepare data
predictor.load_data()
df = predictor.data

# Display processed data
print("\nProcessed Data with Features:")
features = (
    [
        # "Market_State",
        "Close",
        "MA_50",
        # "MA_200",
        "High",
        "Low",
        "MA_7",
        "MA_21",
        "SP500",
        # "TNX",
        "USDCAD=X",
        "Tech",
        "Fin",
        "VIX",
        "Energy",
        # "Fourier_PCA_0",
        # "Fourier_PCA_1",
        # "FT_real",
        # "FT_img",
        # "Market_State",
        # "Market_Sentiment",
    ]
    + [
        "rolling_min",
        "rolling_median",
        "rolling_sum",
        "rolling_ema",
        "rolling_25p",
        "rolling_75p",
    ]
    + ["RSI", "MACD", "ATR", "Upper_Bollinger", "Lower_Bollinger"]
    + ["VWAP"]
    + [  # "Volatility",
        "Daily Returns",
        "Williams_%R",
        "Momentum_Interaction",
        # "Volatility_Adj_Momentum",
        "Stochastic_%K",
        "Stochastic_%D",
        "Momentum_Score",
    ]
)

print(df[features].tail())
print(df[features].shape)


In [ ]:
from stock_prediction.core.models import ARIMAXGBoost
from lightgbm import LGBMRegressor 
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from stock_prediction.utils import optimize_lookback
# Scale the features
lookback = optimize_lookback(
    df[features],
    df["Close"],
    model=XGBRegressor(
        n_estimators=20,
        max_depth=3,
        learning_rate=0.1,
        random_state=42,
        n_jobs=-1,
    ),
    min_window=60,
    step_size=2,
    n_splits=5,
    cross_val=True,
)

In [ ]:
lookback

In [ ]:
predictor.data = predictor.data.iloc[-lookback:, :]
df = predictor.data

In [ ]:
warnings.filterwarnings("ignore")
# seed_everything(42)  # Add at the VERY TOP of the notebook
# Prepare models 
horizon = 20  # 3-week forecast
predictor.prepare_models(
    predictors=features,
    horizon=horizon,
    refit=False,  # Note the metrics are the value of the first fit, not the refit
)

In [ ]:
# Visualize key technical indicators
plt.figure(figsize=(15, 10))

# Price and Moving Averages
plt.subplot(3, 1, 1)
plt.plot(df["Close"], label="Close Price")
plt.plot(df["MA_50"], label="50-day MA")
plt.plot(df["MA_200"], label="200-day MA")
plt.title(f"{symbol} Price and Moving Averages")
plt.legend()

# Momentum Indicators
plt.subplot(3, 1, 2)
plt.plot(df["RSI"], label="RSI", color="purple")
plt.plot(df["Momentum_Score"], label="Momentum Score", color="orange")
plt.axhline(70, linestyle="--", color="red")
plt.axhline(30, linestyle="--", color="green")
plt.title("Momentum Indicators")
plt.legend()

# Volatility
plt.subplot(3, 1, 3)
plt.plot(df["Volatility"], label="Volatility", color="brown")
plt.plot(df["ATR"], label="ATR", color="blue")
plt.title("Volatility Measures")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

# Assuming df contains your stock data with columns ['Date', 'Stock_Close']
try:
    data_FT = predictor.data.reset_index()[["Date", "Close"]]
except KeyError:
    data_FT = predictor.data.reset_index()[["Datetime", "Close"]]
close_fft = np.fft.fft(np.asarray(data_FT["Close"].tolist()))
# Create a DataFrame to store Fourier Transform components
fft_df = pd.DataFrame({"fft": close_fft})
fft_df["absolute"] = fft_df["fft"].apply(lambda x: np.abs(x))
fft_df["angle"] = fft_df["fft"].apply(lambda x: np.angle(x))
# Plot the inverse Fourier Transforms with different numbers of components
plt.figure(figsize=(14, 7), dpi=100)
fft_list = np.asarray(fft_df["fft"].tolist())
for num_ in [3, 6, 9, 50]:
    fft_list_m10 = np.copy(fft_list)
    fft_list_m10[num_:-num_] = (
        0  # Zero out all but the first and last 'num_' components
    )
    plt.plot(
        np.fft.ifft(fft_list_m10), label=f"Fourier transform with {num_} components"
    )
    print(np.real(np.fft.ifft(fft_list_m10)).shape)

plt.plot(data_FT["Close"], label="Real")
plt.xlabel("Days")
plt.ylabel("Stock Price")
plt.title("Stock Prices & Fourier Transforms")
plt.legend()
plt.show()

In [ ]:
# Reassign the Horizon variable      
# Make sure prediction_horizon ≤ prepare_models_horizon to avoid data leakage
horizon = 5

In [ ]:
model_type_1 = "linear"
model_type_2 = "arimaxgb"
# Generate predictions
forecast, backtest, raw_forecast, raw_backtest = predictor.one_step_forward_forecast(
    predictors=features, model_type=model_type_1, horizon=horizon
)

# Generate predictions
forecast_arimaxgb, backtest_arimaxgb, raw_forecast_arimaxgb, raw_backtest_arimaxgb = (
    predictor.one_step_forward_forecast(
        predictors=features, model_type=model_type_2, horizon=horizon
    )
)

In [ ]:
backtest

In [ ]:
predictor.interval

In [ ]:
# Visualize predictions vs actual
from datetime import datetime
plt.figure(figsize=(15, 7))
if predictor.interval == "1d":
    first_day = df.index >= pd.to_datetime(date.today()) - pd.Timedelta(days=2*horizon) # Closer to today
elif predictor.interval == "1m":
    first_day = df.index.hour == datetime.today().hour # Closer to today
# Training period
plt.plot(df["Close"][first_day], label="Historical Prices", color="dimgray", alpha=0.8)

# Backtest period
backtest_start = df.index[-df.shape[0]]
if (backtest["Close"] > 0).all():
    plt.plot(
        backtest["Close"][first_day], label="Backtest", color="blue", linestyle="--"
    )
plt.plot(
    backtest_arimaxgb["Close"][first_day],
    label="Backtest_arimaxgb",
    color="darkblue",
    linestyle="--",
)

# Forecast period
forecast_dates = forecast.index[-horizon:]
if (backtest["Close"] > 0).all():
    plt.plot(
        forecast_dates,
        forecast["Close"][-horizon:],
        label="Forecast",
        color="green",
        linewidth=2,
    )
plt.plot(
    forecast_dates,
    forecast_arimaxgb["Close"][-horizon:],
    label="Forecast_arimaxgb",
    color="darkgreen",
    linewidth=2,
)


# Raw model predictions
if (backtest["Close"] > 0).all():
    plt.plot(
        forecast_dates,
        raw_forecast["Close"][-horizon:],
        label="Raw Forecast",
        color="lightgreen",
        linestyle=":",
    )
plt.plot(
    forecast_dates,
    raw_forecast_arimaxgb["Close"][-horizon:],
    label="Raw Forecast arimaxgb",
    color="green",
    linestyle=":",
)

plt.title(f"{symbol} Price Prediction ({horizon}-day Forecast)")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.show()


# Visualize the log returns
plt.figure(figsize=(15, 7))
real_backtest_area = backtest_arimaxgb.index >= backtest.index[-horizon-1]
plt.plot(
    np.log(df["Close"] / df["Close"].shift(1))[real_backtest_area],
    label="Log Return",
    color="purple",
)
plt.plot(
    np.log(backtest_arimaxgb["Close"] / backtest_arimaxgb["Close"].shift(1))[
        real_backtest_area
    ],  #  backtest.index[-horizon] is the start of the backtest
    label="Log Return Backtest Arimaxgb",
    color="orange",
    linestyle="--",
)
plt.plot(
    np.log(backtest["Close"] / backtest["Close"].shift(1))[
        real_backtest_area
    ],  #  backtest.index[-horizon] is the start of the backtest
    label="Log Return Backtest Linear",
    color="blue",
    linestyle="--",
)

plt.axhline(0, linestyle="--", color="red")
plt.title(f"{symbol} Log Returns")
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.xlabel("Date")
plt.ylabel("Log Return")
plt.show()

In [ ]:
df[['Close', 'ATR']]

In [ ]:
# Model Evaluation
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

# Align dates for evaluation
eval_start = backtest.iloc[-horizon:].index[0]  # backtest.index[0]
actual = df["Close"][df.index >= eval_start]
backtest_eval = backtest["Close"][backtest.index >= eval_start]
backtest_arimaxgb_eval = backtest_arimaxgb["Close"][
    backtest_arimaxgb.index >= eval_start
]


# Calculate metrics
mape = mean_absolute_percentage_error(actual, backtest_eval)
rmse = np.sqrt(mean_squared_error(actual, backtest_eval))
r2 = r2_score(y_true=actual, y_pred=backtest_eval)
bic = predictor.data.shape[0] * np.log(
    np.sum((actual - backtest_eval) ** 2) / predictor.data.shape[0]
) + predictor.data.shape[1] * np.log(predictor.data.shape[0])
aic = (
    predictor.data.shape[0]
    * np.log(np.sum((actual - backtest_eval) ** 2) / predictor.data.shape[0])
    + predictor.data.shape[1] * 2
)

mape_arimaxgb = mean_absolute_percentage_error(actual, backtest_arimaxgb_eval)
rmse_arimaxgb = np.sqrt(mean_squared_error(actual, backtest_arimaxgb_eval))
r2_arimaxgb = r2_score(y_true=actual, y_pred=backtest_arimaxgb_eval)
bic_arimaxgb = predictor.data.shape[0] * np.log(
    np.sum((actual - backtest_arimaxgb_eval) ** 2) / predictor.data.shape[0]
) + predictor.data.shape[1] * np.log(predictor.data.shape[0])
aic_arimaxgb = (
    predictor.data.shape[0]
    * np.log(np.sum((actual - backtest_arimaxgb_eval) ** 2) / predictor.data.shape[0])
    + predictor.data.shape[1] * 2
)

# Print evaluation metrics
print(f"\nModel Performance Evaluation ({len(actual)} days):")
print(f"RMSE Linear: {rmse:.2f}")
print(f"MAPE Linear: {mape:.2%}")
print(f"BIC Linear: {bic:.2f}")
print(f"AIC Linear: {aic:.2f}")
# print(f"R2: {r2:.2f}")
print(
    "if first observation has the correct direction: ",
    np.sign(actual.iloc[0] - actual.iloc[1])
    == np.sign(backtest_eval.iloc[0] - backtest_eval.iloc[1]),
)
print(
    "if second observation have the correct direction: ",
    np.sign(actual.iloc[0] - actual.iloc[1])
    == np.sign(backtest_eval.iloc[0] - backtest_eval.iloc[1]),
)
print(
    "if last observation has the correct direction: ",
    np.sign(actual.iloc[-1] - actual.iloc[-2])
    == np.sign(backtest_eval.iloc[-1] - backtest_eval.iloc[-2]),
)
print(
    "if last second observation have the correct direction: ",
    np.sign(actual.iloc[-2] - actual.iloc[-3])
    == np.sign(backtest_eval.iloc[-2] - backtest_eval.iloc[-3]),
)
print("")

print(f"RMSE Arimaxgb: {rmse_arimaxgb:.2f}")
print(f"MAPE Arimaxgb: {mape_arimaxgb:.2%}")
print(f"BIC Arimaxgb: {bic_arimaxgb:.2f}")
print(f"AIC Arimaxgb: {aic_arimaxgb:.2f}")

print(
    "if first observation has the correct direction: ",
    np.sign(actual.iloc[0] - actual.iloc[1])
    == np.sign(backtest_arimaxgb_eval.iloc[0] - backtest_arimaxgb_eval.iloc[1]),
)
print(
    "if second observation have the correct direction: ",
    np.sign(actual.iloc[0] - actual.iloc[1])
    == np.sign(backtest_arimaxgb_eval.iloc[0] - backtest_arimaxgb_eval.iloc[1]),
)
print(
    "if last observation has the correct direction: ",
    np.sign(actual.iloc[-1] - actual.iloc[-2])
    == np.sign(backtest_arimaxgb_eval.iloc[-1] - backtest_arimaxgb_eval.iloc[-2]),
)
print(
    "if last second observation have the correct direction: ",
    np.sign(actual.iloc[-2] - actual.iloc[-3])
    == np.sign(backtest_arimaxgb_eval.iloc[-2] - backtest_arimaxgb_eval.iloc[-3]),
)
# print(f"R2 Arimaxgb: {r2_arimaxgb:.2f}")

In [ ]:
# Feature Importance Analysis
from stock_prediction.utils import feature_importance

if hasattr(predictor, "feature_importances"):
    print("\nFeature Importances:")
    # importance_df = pd.DataFrame(predictor.feature_importances).T
    # importance_df = importance_df.sort_values(by='Close', axis=1, ascending=False)
    importance_df = feature_importance(
        predictor.data.drop(columns="Close"), predictor.data["Close"]
    )
    display(importance_df)

    importance_df.plot("Feature", "Importance", kind="barh", figsize=(20, 15))
    # plt.xticks(fontsize=5)
    # plt.yticks(fontsize=5)
    plt.title("Feature Importances for Close Price Prediction")
    plt.show()

In [ ]:
# Feature Importance Analysis
from stock_prediction.utils import feature_importance

if hasattr(predictor, "feature_importances"):
    print("\nFeature Importances:")
    # importance_df = pd.DataFrame(predictor.feature_importances).T
    # importance_df = importance_df.sort_values(by='Close', axis=1, ascending=False)
    importance_df_features = feature_importance(
        predictor.data[backtest.columns].drop(columns="Close"), predictor.data["Close"]
    )
    display(importance_df_features)

    importance_df_features.plot("Feature", "Importance", kind="barh", figsize=(20, 15))
    # plt.xticks(fontsize=5)
    # plt.yticks(fontsize=5)
    plt.title("Feature Importances for Close Price Prediction on features")
    plt.show()

In [ ]:
# Prediction Details
print("\nDetailed Forecast:")
if "Momentum_Score" in forecast.columns:
    display(forecast[["Close", "Momentum_Score"]].tail(horizon))

print("\nKey Statistics:")
print(f"Forecast Range: {forecast_dates[0].date()} to {forecast_dates[-1].date()}")
print(f"Predicted Change: {(forecast['Close'].iloc[-1]/df['Close'].iloc[-1]-1):.2%}")
print(f"Predicted Change (Arimaxgb): {(forecast_arimaxgb['Close'].iloc[-1]/df['Close'].iloc[-1]-1):.2%}")
if "Momentum_Score" in forecast.columns:
    print(f"Average Momentum Score: {forecast['Momentum_Score'].mean():.2f}")
print(f"Average Momentum Score (Arimaxgb): {forecast_arimaxgb['Momentum_Score'].mean():.2f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# Compute residuals
residuals = backtest[-horizon:].Close - predictor.data[-horizon:].Close

# 1️⃣ Histogram & KDE Plot
plt.figure(figsize=(8, 5))
sns.histplot(residuals, kde=True, bins=30)
plt.axvline(x=0, color="red", linestyle="--")  # Zero reference line
plt.title("Residual Distribution")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

# 2️⃣ QQ Plot (Quantile-Quantile Plot)
plt.figure(figsize=(6, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("QQ Plot of Residuals")
plt.show()

# 3️⃣ Statistical Normality Test (Shapiro-Wilk)
shapiro_test = stats.shapiro(residuals)
print(
    f"Shapiro-Wilk Test: W={shapiro_test.statistic:.4f}, p-value={shapiro_test.pvalue:.4f}"
)

# Interpretation
alpha = 0.05  # Significance level
if shapiro_test.pvalue > alpha:
    print("Residuals appear to be normally distributed (Fail to reject H0).")
else:
    print("Residuals are NOT normally distributed (Reject H0).")

In [ ]:
# y_true_before_backtest = predictor.data[raw_backtest.columns].iloc[-horizon-1].values # Get the last row of the data before backtest

# y_pred_first_backtest = raw_backtest.iloc[-horizon].values # Get the first row of the forecast data of original backtest
# y_true_first_backtest = predictor.data[raw_backtest.columns].iloc[-horizon].values

# y_pred_last_backtest = raw_backtest.iloc[-1].values # Get the first row of the forecast data of original backtest
# y_true_last_backtest = predictor.data[raw_backtest.columns].iloc[-1].values


# # Compute Up (1) or Down (0) movement of ALL Features
# y_true_1d = (y_true_before_backtest > y_true_first_backtest).astype(int)  # 1-day movement
# y_pred_1d = (y_true_before_backtest > y_pred_first_backtest ).astype(int)
# y_true_horizon = (y_true_before_backtest > y_true_last_backtest).astype(int)
# y_pred_horizon = (y_true_before_backtest > y_pred_last_backtest).astype(int)


y_true = np.sign(
    predictor.data[raw_backtest.columns].iloc[-horizon - 1 :].Close.diff().dropna()
)  # Convert to Up/Down movement
y_pred = np.sign(
    raw_backtest.iloc[-horizon - 1 :].Close.diff().dropna()
)  # Convert predictions to Up/Down

## Backtester and StressTester


In [ ]:
import pandas_market_calendars as mcal
import pandas as pd
# nyse = mcal.get_calendar("NYSE").schedule(
#    start_date="2020-01-01", end_date="2023-12-01"
# ).index

In [ ]:
symbol = "CRWD"

In [ ]:
from stock_prediction.core import Backtester, StockPredictor

predictor = StockPredictor(symbol, start_date="2023-06-15", interval="1d")
predictor.load_data()
# backtester = Backtester(predictor)
# print(backtester.portfolio)
# history, report = backtester.run_backtest("2023-01-03", "2023-01-09")
# print(backtester.portfolio)

In [ ]:
# from stock_prediction.core import StockPredictor
# StockPredictor.create_hqm_stocks(start_date= "2021-01-01", end_date="2023-01-01").head(10).Symbol.tolist()

In [ ]:
bt = Backtester(predictor)
start_date = "2025-01-01"
end_date = "2025-05-05"
# Run backtest
history, report = bt.run_backtest(
    start_date=start_date,
    end_date=end_date,
)

In [ ]:
bt.portfolio

In [ ]:
print('Number of BUY:',len([order for order in bt.portfolio["transactions"] if order[0] == "BUY"]))

print('Number of SELL:', len([order for order in bt.portfolio["transactions"] if order[0] == "SELL"]))
nyse = mcal.get_calendar("NYSE").schedule(
    start_date=start_date, end_date=end_date
).index
print('Number of trading days:', len(nyse))

In [ ]:
history

In [ ]:
(history['value'].diff().dropna()>0).astype(int).sum()/report['num_trades']

In [ ]:
report
# save the report to a txt file in the dir


In [ ]:
from matplotlib import pyplot as plt
import yfinance as yf
from stock_prediction.utils import  get_next_valid_date
plt.figure(figsize=(15, 7))
plt.plot(yf.download(symbol, start=start_date, end=get_next_valid_date(end_date), interval='1d').Close)
plt.title(f"{symbol} Price")
plt.xlabel("Date")
plt.ylabel("Price")

plt.figure(figsize=(15, 7))
plt.plot(
history['value'])
plt.title("Portfolio Value Over Time")
plt.xlabel("Date")
plt.ylabel("Portfolio Value")
plt.show()

In [ ]:
import yfinance as yf
# Load data into a DataFrame
df = pd.DataFrame({
    'Portfolio_Return': history['value'].pct_change().dropna().values,
   'Stock_Return': yf.download(symbol, start=start_date, end=get_next_valid_date(end_date), interval='1d')["Close"].pct_change().dropna()[symbol].values
})

# Calculate covariance matrix
covariance_matrix = df.cov()
covariance = covariance_matrix.iloc[0, 1]

# Calculate variance of stock returns
variance = df['Stock_Return'].var()

# Beta
beta = covariance / variance
print(beta)  # Output: ~0.59

In [ ]:

import yfinance as yf
import pandas as pd

# de-annualize yearly interest rates
def deannualize(annual_rate, periods=365):
    return (1 + annual_rate) ** (1/periods) - 1

def get_risk_free_rate():
    # download 3-month us treasury bills rates
    annualized = yf.download("^IRX")["Close"]
    
    # de-annualize
    daily = annualized.apply(deannualize)

    # create dataframe
    return pd.DataFrame({"annualized": annualized["^IRX"].values, "daily": daily["^IRX"].values})    


In [ ]:
# risk-free rate 
get_risk_free_rate()

In [ ]:
report["total_return"]

In [ ]:
# from stock_prediction.core import StressTester
# stress_tester = StressTester(predictor)
# stress_tester.portfolio
# stress_history, stress_report = stress_tester.run_stress_test(start_date=start_date, end_date=end_date)

In [ ]:
# stress_history['value'].plot()